In [ ]:
!pip install ortools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 27.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from geopy.distance import geodesic

def create_data_model(starting_point, bins):
    data = {}

    # Adding the starting point to the bins
    bins.insert(0, (starting_point, 0, 0, 0))  # Assuming (0, 0) as a placeholder for the starting point

    # Creating the distance matrix
    data['distance_matrix'] = [
        [geodesic((lat1, lon1), (lat2, lon2)).km for _, lat2, lon2, _ in bins] for _, lat1, lon1, _, in bins
    ]

    data['num_vehicles'] = 1
    data['depot'] = 0

    return data

def tsp_solver(starting_point, bins):
    data = create_data_model(starting_point, bins)

    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        return data['distance_matrix'][manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.time_limit.seconds = 1  # Set a time limit for the solver, adjust as needed

    solution = routing.SolveWithParameters(search_parameters)
    index = routing.Start(0)

    route = []
    while not routing.IsEnd(index):
        route.append(manager.IndexToNode(index))
        index = solution.Value(routing.NextVar(index))

    return route

# Example usage with sample coordinates
starting_point = "Start"
bins = [
    ("Bin1", 37.7749, -122.4194, 1),  # San Francisco, CA
    ("Bin2", 34.0522, -118.2437, 2),  # Los Angeles, CA
    ("Bin3", 41.8781, -87.6298, 3),   # Chicago, IL
    ("Bin4", 40.7128, -74.0060, 4),   # New York, NY
    ("Bin5", 51.5074, -0.1278, 5)     # London, UK
]

optimized_route = tsp_solver(starting_point, bins)
print("Optimized Route:", optimized_route)


Optimized Route: [0, 5, 4, 3, 2, 1]
